# Классификатор DGA доменов

## Ссылки:

- https://habr.com/ru/companies/pt/articles/282349/
- https://www.securitylab.ru/blog/personal/Morning/347957.php
- https://underdefense.com/guides/detecting-dga-domains-machine-learning-approach/
- https://habr.com/ru/companies/wunderfund/articles/331310/
- https://github.com/ryancollingwood/DGAClassifier

## Наборы данных

- https://www.kaggle.com/datasets/gtkcyber/dga-dataset
- https://github.com/chrmor/DGA_domains_dataset


https://github.com/keras-team/keras/issues/18913#issuecomment-2001958300

In [ ]:
!nvidia-smi

In [24]:
!pip install tensorflow==2.15.1
!pip install keras==2.15.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 32.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 30.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Input, Bidirectional
from keras.layers import LSTM

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

dir_path = 'work/models/dga/'

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [3]:
!pip install pycaret
!pip install mlflow
%pip install scipy==1.11.4

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
df1 = pd.read_csv('work/datasets/dga_data.csv')
df2 = pd.read_csv('work/datasets/dga_domains_full.csv')
df3 = pd.read_csv('work/datasets/dga_domains.csv')

In [5]:
## Очистка и преобразование исходных данных
df1.loc[(df1['isDGA'] == 'dga'), 'isDGA'] = 1
df1.loc[(df1['isDGA'] == 'legit'), 'isDGA'] = 0

df1 = df1.drop(columns=df1.columns[1], axis=1)
df1 = df1.rename(columns={'host': 'domain'})

df2.loc[(df2['isDGA'] == 'legit'), 'isDGA'] = 0
df2.loc[(df2['isDGA'] == 'dga'), 'isDGA'] = 1

df3.loc[(df3['isDGA'] == 'legit'), 'isDGA'] = 0
df3.loc[(df3['isDGA'] == 'dga'), 'isDGA'] = 1

df3 = df3.drop(columns=df1.columns[2], axis=1)
df3 = df3.drop(columns=df1.columns[1], axis=1)

df3.head()

df3 = df3.rename(columns={'host': 'domain'})

In [6]:
df = pd.concat([df1, df2, df3])

df = df.astype({'domain': 'string', 'isDGA': 'int8', 'subclass': 'category'})

In [7]:
# df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 968824 entries, 0 to 133925
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   isDGA     968824 non-null  int8    
 1   domain    968824 non-null  string  
 2   subclass  834898 non-null  category
dtypes: category(1), int8(1), string(1)
memory usage: 16.6 MB


In [8]:
X, y = df['domain'], df['isDGA']

In [9]:
ascii_ranges = ["_", "-", ".", "0", "1", "9", "a", "z"]
ascii_ord_ranges = list()
for c in ascii_ranges:
    ascii_ord_ranges.append(ord(c))

expected_ords = [ord("-"), ord("_"), ord(".")] + list(range(ord("0"), ord("9") + 1)) + list(
    range(ord("a"), ord("z") + 1))

for i in ascii_ord_ranges:
    assert (i in expected_ords)

ord_data_dfs = dict()

for column in ['domain']:
    ord_data_dfs[column] = df[column].apply(lambda x: [ord(w) for w in x.lower()]).apply(pd.Series)
    display(ord_data_dfs[column].head())

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
0,54.0,120.0,122.0,120.0,115.0,119.0,51.0,115.0,111.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103.0,108.0,98.0,116.0,108.0,120.0,119.0,119.0,104.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120.0,120.0,109.0,97.0,109.0,111.0,112.0,121.0,105.0,112.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,122.0,102.0,100.0,53.0,115.0,122.0,112.0,105.0,49.0,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,106.0,112.0,113.0,102.0,116.0,121.0,109.0,105.0,117.0,118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for column in ['domain']:
    ord_data_dfs[column] = ord_data_dfs[column].fillna(-1)

In [18]:
unknown_ord_columns = dict()

for column in ['domain']:
    unknown_ord_columns[column] = set()

    for ord_column in ord_data_dfs[column].columns:
        try:
            assert (all(ord_data_dfs[column][ord_column].isin(expected_ords + [-1])))
        except AssertionError:
            print(column, "- column:", ord_column, "contained unknown ordinal value")
            unknown_ord_columns[column].add(ord_column)
            continue

    print()

for column in ['domain']:
    for ord_column in unknown_ord_columns[column]:
        unknown_ord_index = (~ord_data_dfs[column][ord_column].isin(expected_ords + [-1]))
        if any(unknown_ord_index):
            print(ord_data_dfs[column].loc[unknown_ord_index][ord_column])
            display(df.loc[unknown_ord_index][column])

    print()

In [19]:
validTokens = {x: idx + 1 for idx, x in enumerate(set(''.join(X)))}

maxFeatures = len(validTokens) + 1
maxLength = np.max([len(x) for x in X])

TypeError: sequence item 0: expected str instance, list found

In [ ]:
X = [[validTokens[y] for y in x] for x in X]
X = pad_sequences(X, maxlen=maxLength)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36)

In [23]:
# LSTM модель
model = Sequential()
model.add(Input((73,)))
model.add(Embedding(maxFeatures, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [22]:
model.fit(X_train, y_train, batch_size=15, epochs=5)

NameError: name 'model' is not defined

In [21]:
# https://www.tensorflow.org/guide/keras/serialization_and_saving
model.save(dir_path + 'dga_lstm.keras')

NameError: name 'model' is not defined

In [ ]:
# model = keras.models.load_model(path)

In [ ]:
predictions = model.predict(X_test).round()

In [ ]:
acc = accuracy_score(y_test, predictions)
classification_report = classification_report(y_test, predictions)

In [ ]:
print(classification_report)

In [ ]:
def tokenize_domain(domain):
    d_ = [[validTokens[char] for char in domain]]
    d_ = pad_sequences(d_, maxlen=maxLength)

    return d_

In [ ]:
model.predict(tokenize_domain('example.com'))

In [ ]:
model.predict(tokenize_domain('cvyh1po636avyrsxebwbkn7.ddns.net'))

In [ ]:
validTokens

# PyCaret example

In [ ]:
from pycaret.nlp import *

nlp_df = setup(df, target='domain', session_id=126, log_experiment=False, log_plots=True, experiment_name='dga_nlp_1')

In [ ]:
s = ClassificationExperiment()
s.setup(df, target='is_legit', session_id=123, use_gpu=True, log_experiment=False,
        experiment_name='semantic_classification')

best_regression_model = s.compare_models()